In [6]:
import cv2
import numpy as np
import requests
import json
import base64
from PIL import Image

import paho.mqtt.client as mqtt
import json

In [ ]:
def main():
    # 選擇第二隻攝影機
    cap = cv2.VideoCapture(0)

    # HSV red filter
    lower = np.array([155, 150, 150])
    upper = np.array([180, 255, 255])

    client = mqtt.Client()
    # 設定登入帳號密碼
    client.username_pw_set("", "")

    # 設定連線資訊(IP, Port, 連線時間)
    client.connect("140.124.73.217", 1883, 60)

    # area = 150
    wait = 0
    while True:
        red_count = 0
        # 從攝影機擷取一張影像
        _, frame = cap.read()
        orig_frame = frame.copy()
        img = cv2.cvtColor(orig_frame, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(img, lower, upper)

        # Drawing Bounding Boxes Around Detected Objects
        contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        # draw the contours, not strictly necessary
        # for i, cnt in enumerate(contours):
        # cv2.drawContours(frame, contours, i, (225, 225, 255), 3)

        for contour in contours:
            # continue through the loop if contour area is less than area...
            # ... helps in removing noise detection
            # if cv2.contourArea(contour) < area:
            # continue

            # get the xmin, ymin, width, and height coordinates from the contours
            (x, y, w, h) = cv2.boundingRect(contour)
            # draw the bounding boxes
            cv2.rectangle(orig_frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            red_count += 1

        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(orig_frame, str(red_count), (10, 30), font, 1, (0, 255, 0), 2, cv2.LINE_AA)

        # 顯示圖片
        cv2.imshow('Detected Objects', orig_frame)
        # cv2.imshow('frame', frame)
        cv2.imshow('mask', mask)
        # cv2.imshow('img', img)

        # output image
        if wait < 100:
            wait += 1
        else:

            # base64 encode
            img_resize = cv2.resize(orig_frame, (80, 60))
            cv2.imwrite(r'C:\Users\asus\Desktop\MmsLab\Project_DetectRedDot\bb_img.jpg', img_resize)

            img_base64 = base64.b64encode(cv2.imread(r'C:\Users\asus\Desktop\MmsLab\Project_DetectRedDot\bb_img.jpg')).decode('utf-8')
            print(img_base64)

            # Send http request
            #url = "http://localhost:4000/red"
            #headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
            #r = requests.post(url, json={'color':'red','value':red_count,'img':img_base64}, headers=headers)
            #print(f"Status Code: {r.status_code}, Response: {r.json()}")

            #mqtt publish
            #payload = {'color':'red','value':red_count,'img':img_base64}
            #client.publish("Try/MQTT", json.dumps(payload))

            #print(final_data)
            wait = 0
        #print(wait)

        # 若按下 q 鍵則離開迴圈
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # 釋放攝影機
    cap.release()
    # 關閉所有 OpenCV 視窗
    cv2.destroyAllWindows()


if __name__ == '__main__':
    main()

//7/+v/77v/x7f/y+v/8/v3///z+////+P/8+P/+z9XUztPUxcvQzNXZ0tvf2eXn4Orx4+/11+Ho7Pb97fb68vv/+v//+P3/+///+///+///+///+///+///+//++//+///+/f/+/f//+///+v//+v//+///+//////////+/f//+///1tnh5uz31d7sws3br7bFrrbDrrfFqLG/oqy+oqy+nKe9mKO5maO7mKK6l563lJu0lJqxkpivkJSsj5OriYqkhYmigYWdfISbd3+WcXySaHOJWWR6a3WNc32Vb3iTbXaRZG6GYmyEY3CGZ3SK+v/78v/z2v/czO/N8f/0//7///3//v7+9fz59P36+///0tXZw8vSwszTzdjc2OTm3Ofr4u704Orx5/H47PX57/j88vn8+f7/+///+///+///+///+///+///+//+/P/9///+///+/f//+///+/7/+v//+////f/////+/f78////+v7/4eTs6e/61t/twsvZtL3Kr7fErLXDqLG/oau8oKq8m6a6maS6lZ+3lJ62l563lZ20kJatjZOqjJCoiIykiImjh4ukgoife4OadX2UcHuRZXCGUl1zYWuDZnCIU1x3XGZ+VF52W2Z8X2yCZXKI7P/uo82k3P/beK928//y/f/+//7///7/+v/90NXT/Pr/4eDqx83YwMrUxtLWztrc2eTo2+Xs3ujv0Nrh5u/z7fb68vn88vn8+P3++f7/+///+v7//P7+/f///f///P7+///+///+/f//+///+///+////f///f/////+/v/9/////f//9Pf89fr/3eXyx9DeuMHOrrfEqrPBp7C+oqy9n6m7mqW5mqW5l5+2maG4lZ20kJivkZeujJGmhough4ujgoafhIihgIadeoKZc3uSbnmPZ3SKWmd9VF52WmR8XWd/ZW+HY2+HWGV7V2R6V2V47P/v6P/q0P/OxPXB9P/0/P/9//7///7//P/9+///

+/7/+P/+8//09P/1+//+/v3///z+//7//f39/P7+z9TSztbVvs7Uw9bbz9vf2ufp3evx3u700+Ll5/b57fj87/r+9/7/9/7/+///+///+///+////f///P7+/f/////////////////+/f/++///+////f7//f////7///39///+/f//4+bry8/ay9Ljz9bpvMTRsrvFr7fEq7TCpq6/oqy+nai+m6a8nae/mKK6lqC4k521kpqxjJSrk5uyjpSrj5GphoaeiIqihYmhfoSbd3+Wc36UYm2DWGN5anWLbXiMbHeLV2V4TFptVmV4YG+C9v/78f/z3v/dz+7P8/70/f///v3//v7+9fj2+//++/39z9TXwtDWu8vRy9ja2OTm2ejr3evx4vH04Ozw6PP39P3/8Pf69v3/+v//+v7/+///+////f///P7+/P7+/////////////f/++//++///+///+////f///////f39///+/f//7fH23OHq1dzt0tnsvMTRsbrErbXCqbLAo62+oau9m6a6maS6l6G5lJ62k521kpy0lJyzkJivkZmwiI6lh4mhg4WdhYaggYWdf4Sdd3+WcXuTaHOJVF52XGd9Xml/WmV7XWqAWmh7U2F3WGd66v/uo86j3P/YeLB18f/y+v/+/f///f/++P/51NvW/Pr/3t3nydHYwMzQyNLS0dvb3Ojs2ubs3ent0Nvf5/L26/T47/b58vn8+P3/+f7/+v7/+////f///f///f///f///////////f/++//++v//+v//+v//+////f//+/39/f/++//+9v3/7fT93OT10dnqv8bVs7vIrbbEqbLAo62+oau9nqe7m6S4maG4mqK5m6O6lJyzj5Wsi5GojZSoh42khoehh4iihYaifoKbfYGdcXiRaXKNanSMa3SPWmR8XGZ+X2mBZHCIZXKIXmqCTVpw7P/t6P/n0P/KxPe+8f/y+f/7/f/+/f/8+v/79/75

+///+P/78v/x8//x+//8/v7+//7+///+/f78/f/+1NfVzNTTwNHUv9LV0t7i2+fp4uzz4+303OXp7PX57vT59vz/9/r++/7//f///f///f///f///f///f///f///f/+///+///8////+///+/7/+v//+////f/////////+//////7/+vz/yc3Y3uPy1Nvqz9blv8fUsrvJsbrIqrHEqbDDo6vCpKzDm6W9nKa+maW9kp62lqK6kp62jZmxi5ask5arjY6igoaZgoibgIWaeH+TdX6Sb3iMY26CV2J2aHSGaHSGZnODVmNzTltrV2R09v/78f/z3v/cz+7N8//z/f///v7+/v/99/r4+/78/f//zdLVxNLYucnPydXZ1+Pl2uXp5/H43ejs6fL27/j88ff8+f7/+f7/+///+///+///+////f///f///f///f/+///+///8/f//+///+v//+v//+///+//////+///+/////v3/+v3/19zl5uv6197tzdXiu8PQs7zKrbbEq7PEpq3Ao6zAoanAnKa+mqS8l6G5mKS8l6O7jpqyiZWtkZmwjJGmi46jhIecfYKXeYCUeH+Tc3uSa3SIYWyCT1puWWR4XGh6W2d5W2d5VGByWWZ27f/ups6l3//Zeq928//y+v/9/f///P7++P/6ytHO/P3/2NnjydLcxdDYw87S0d3f2OPn2+ft3+vvztnd5O/z7/j87vf67/b5+P3/+f7/+v//+v//+///+///+////f/+///+///8/f//+///+v//+P//+v//+////f/+/f/8///++/39+v//6PD37vb/2eLv0drnv8jVt8DOr7jGrLTFpay/oKm9naa6mqK5maS6laC2mKO5kp2zjpmvh5KojJSrhYmhhYefg4WdeX2VdHqRdnyTcXiRbHSLaHKKbXiOV2J4VF9zX2p+aHOHXWh8WmZ47v/v6v/p0//OxfXB8//y+v/9/////f//+v/8+P/8

+v//9v/78v/08//1+//8/v7+//38///++/r8//7/09XWzNHSwdDSxdbZz9ng2+Tt4evy5fH32OLp6PL58Pb7+P7/+fz/+/7//f///f///////////////////f/+///+///8///8/////f///f7/+////f///f/+///+///7///+/f//+fz/zNLZ4Onz0drnzdTjucDPsrrLrrbHqrTGprDCoq3Dn6rAmqa+mqa+lqK6lqK6laC2kZyykJuxi5OqkZSpkZKniYyhgoecgIade4Oac36UbXiOYGuBZXCGbXiOdH+VbHqNZXOGX2+AYnKD9P/47//y3v7fz+3Q8/70/f/+/v/9/v/9+Pr7/P3///3/0dTZw9DSvczPzdfe1+Dp2+ft5/P54+305vD37PX59fv/+P3/9/z/+v7/+////f///f///f///f///f/+/P/9///8///8/////f//+///+///+////f/+///+///8///+/f//+v3/3uTr7PX/2+TxztXku8LRtLzNrrbHqbPFpa/BoazAnqm/narAl6S6lKC4l6S6l6K4kZyyj5qwjpatk5ari46jh4qfhImef4WceYGYcn2Ta3aMX2qAWGN5b3qQY26ES1lsU2F0TV1uUmJz7P/ro86j3f/deq178f/y+v/8/f/8/f/++f3+zM/T/fn/3dzmydHYw8/TyNDX0Nrh2eXr1uTq2ubs1N7l5vH16vP38vv+8fj79vv8+f7/+///+///+v/++///+////f/+///8///8/f//+///+v//+v//+v//+//+/f/+/f/8/P/9+///+v//7/j88/7/4Or0zNThvMTRtb7MrrfFprDBoqy+oKu/nKe7mqW5maS4mqW7lqG1k5ywkZqujperjpWpi5Clh4qfhYidfoWZdn6VdoGXc36UZXCGZXCGVWB2UFtxVmF3ZHKFYW+CUWFyVmZ37v/s6v/n0v/RxfPE8//0+v/7///8///++v7/+/7/

////+v/78v/x8f/y+//7/v7+/v7+/f//9/z6+v/+y9LPytXTxtLYxNLY0t3h3ebp4Orx5vD32eHo7fX88ff88ff8+/7/+fz//f///f///f///f///f///f///f/+///+//////////7/+///+///+v///f///f////////////7///7/+fz/1Nrf4uny1+DqztfkuMHPtr7Pr7fIrrXJq7LGpqzDpavCoajBnaS9mKG8m6S/kp62j5uziZWtipSsjZOqiIykg4efhYmhgIWefIGad3yVd3yVYmmCbHOMbHSLc3uScHmNbHWJa3WHZG6A+P/58v/y3v/cz+7N8//z/f///v7+/P7+9Pn3+v/9+///zNPWws7UvszSz9re1eHj2uTr4uzz5e/25Ozz6/T49fv/9Pn8+f7/+///+///+///+////f///f///f/+/f/+/////////f//+///+v//+v//+////f////////////3///7/+/7/4ujt7fX83ufwz9jlvcbUt7/QsbnKrLPGqbDEpavCo6nAoqnCnKO8l6G5maO7lZ+3kJy0jZevipWripCnhYmhg4effYGZgIWeeX6Xd3yVb3aPW2J7ZWyFanKJanKJUltvVmF1T1lrXWl76v/upM6l3//YfK918//y+v/+/f///P7+/P/7y87M/vv/29vnxtDawtLZw8/T0t7g3Obt3Oju3urw0tzj6PP36vP37vf68vn8+P3++f7/+v//+v//+///+///+////f///////////f//+///+v//+v//+v//+////P7+/f///v7+/P7/+v//8Pn99P7/4Ovzzdbjv8jVtr/NsrrLp7HDpK7Aoaq+nqe7nKS7m6O6nKS7maG4kpqxiZSqjJSrjJSrhYmhh4qffYGZfoKaeH2We4CZb3aPanGKaG+IT1lxVmF3VF91ZXCEXWt+VWFzWGZ46f/v5v/p0v/MxPW/8//0+v/+//7////////+/f39

...

/f7/+v/88v/x8//x+v/8/P7//v3/////+Pr6/P/90tjTzNTTxNDUyNbc1Nve3ufq3+vv4PD21ePp5/X77/r+7fj8+P//9v3/+///+v7//P7+/f///f/+/f/+/f/+///+///+///+//7//f//+/7/+/7//f///f////////79/////f//1djg4ujz1d7sx9Hiv8rYu8bUt8HSsbvMp7PFo6/Bn6q+mqW5mqW7mqW7lqG3mKO5maK2lp+zlp+zkpyulJ2rho+cf4uXcn6KZ3N/XGh0XWVyZGp3YGNxZ2h2X2BuZGd1Zmx5dH2Kd4OPa3mF9v/78f/z3v/czu/N8f/0/f///v3//v7++fz6+/78/f//z9TXws7Uv8vR0Nnc2eLl3uru4vD25PL44u706PP38vv/8vn8+P//+v//+////f///f///f/+/f/+/f/+///+///+///+/f//+///+/7/+///+////f/////+/v/9/v7+/P7/4eTs5Or10droxtHfvsnXusXTtb/QsLrLqbPFoq7An6i8maS4m6S4laC0lZ20mKG1lp+zl6Gzm6K2kpmsho+dcXqHYm56X2p4VmFvXWh2YGd4XGByXF5wWlxuXFxsW15tWF9ucnuIeoaSeIaS6v/uos6l3P/beK948f/0+v/+//////7+/P/9z9HR//r/3NrmyNDXxNDUx9PV0Nze2OTo2+ft3urw1eHn6PP37PX58Pf69Pv++P3++v7//f///f///f///f/////+///+///+///+/f//+///+v//+v//+v//+////f/+/f/+/f//+///8/n+6/T90t7qy9bkvsnXu8TRtb7MsLnHqbHCoqy9n6a5mKK0m6K1k52vl56ymaCzk5qtk5uslZuuiJCha3SCYm56YGt5eoSVkJqsnaa6oKS8oKS8nZ+3hoigfX6Tam6BX2Z3Z3B+c3+LdYOP6//t6P/p0f/PxPXB8//0/P/9//7///7//f78/f//

//7/+P/+7P/z7f/0+//8//3///z+////+fv7+v/+zdfRydfTxtLWxdbZzN7f1ero4O/y3u3w2uXp7Pf77/j79P3/+v//+P3+/f///P7//v7+/////////v7+///+///+/f///f///f///f/////+///+///+///+/f78///+/f///f7/+fz/1tfhvb3J4uPx1t3uz9foxc/gusTVtsDRr7nKrLjKrLjKqbTKoazCnqjAnae/mKK6lJ62laC2l5+2mZ6zmJ2ykpesjperhpCiiJSmiJaohJKkgI2dfYqafYaUd4CObHOCXmV0Yml4X2Z1+v/88f/12v/dzO7P9f70//7///3///7+9vn3+f78+v//zNTUw87Sws7Sytnb0eHg1eTn5fT34Ozw4+7y7fn78vv+8/v7+v//+///+////f///f///f///f/////+/f/+/f///f///f///f///f/+/f/+///+/f/+///+/P/9/f//+///+v3/293lxcXR5Of12N/uz9fow83euMLTtb/QrbfIqLTGprLEpbDEoKvBnqm/nai+mKK6k560kp2zlZ6ylpuwjpOok5quj5ishpKkg4+hgI6gcX+RaneHY3CAYWt8W2RyWmFyWWBvXWR1XmV07f/wpcym3f/Zeq929P/0//7///7//////v/7zM3L//3/3dzmzc/ZzNLZxc7R0d3d1uXo3ezv2ejr0d3h6PX36PT27fb59Pv+9v7++f7/+///+///+///+////f//+///+//++//++///+///+///+////f/++//+/f/++f78+v/++v//+v7/5Ofv1dfh6u752N/uzdbkwMvZtsHPtL/NrbjGp7TEo6/Bn6u9nai8nKW5mqO3lp61kZquj5islJuukJapi5Gkk5qti5WnhJCifYudcoCSYW+BWGR2U19xWWN1V2FyW2J1V19wUllsVV1u7v/w6v/q0v/NxfXB9v/1/v7+//7////////+/f39

/f7/+v/88P/z8f/0+v/+/P3//v3//f//9/z9+P//y9DRzNXYwtDWwdHX09/l2ubq4e3x4O/y1+bp6Pf67vr88v7/9v3/9/7/+f7/+f7/+///+v7//P7+/P7+/v7+/v/9/v/9/v/7/v7+/v7+/P7//P7/+v7/+v/+/v7+/v/9//3///3/0dDawMPR1dzt093uwczavMjUtcDOr7rIqbXHqbXHprHFoKu/nqm/maS6l6K4kp2zkp2xkZywlZ6ykpuvlJuvkJeqj5mriJKjhpGfhpGfeYOUbniKaHGFXmZ9Wl52X2R5YWV4XmNyWl1rYWVw+P/78f/z3v/cz+7N8/70/f///v3//v3/9Pn4+///+///zdHWxNDWvc3Tzdjc2OTm2ubq4fDz4fDz3uru6/f56/f59P3/9fz/+f7/+v//+///+v7//f///f/////////+///+///8/////f///f//+///+///+////f/+///+//7+//z/1tXeys7Z19/w093uwMzYu8fTtL/Nr7rIqrTFp7PFpK/Dn6q+nKe7l6K2lqG3kZywk5ywkZqukZquj5aqkZislJuujpiqipSlfIeVbHeFXWd4WWN1Vl9zWWF4WmB3XWJ3WV1wXWJxYGNxVlpl7P/tps6k4f/Zfa919P/x/P/9//7///3//v/70NHP//3/3drjyNHaw9HXw83N1d/f2ebo3uru3uru1uHl6vb46fX37/j78fj7+f7/+f7/+v7/+////f///f/////////+/f/+/f/8/f///f///f//+///+v//+v//+//+/f/+/////fz+3+Ln2t/o2+Ty0NvpwMzYvMbQtr/Mr7rIqbPEprDBoqy+n6m7m6W3lqCylZ6ykJqskpmsj5apj5api5KlkZisiZCjhI6gf4maanWDXWh2VmBxcHqMh5CkmKG1n6W8pqvAiY2giI2cXF9tW19q6//s6P/n1f/Nyfa+9v/x/P/7//7///7////8/vz7

//3/+//87v/z7//y+v/+//z///3/////9fv69v/+zNPQy9PSx8/WxtDX0d3f2+fp3ent4u702uXp7Pf78Pn98fr++f7/+f7//f///f///f///P7//P7+/f///f///f/+///+///8//////////7//f7/+///+//////////+//3+//3/zczVvsLN1dzr0t3rw87cu8fTsr/NrrvJqLbIqLbIprXIobDDnavBmKa8lKG3k6C2mKC3l5+2lZuykJivi5Soh5Olf4udfoubg5OgjJypjp2th5amipWpjJerjJOneoGUaG18YmhzWF5lVVtg+v/78v/z3P/czO/N8f/0//7///3///7+9Pv4+P/++///zdLVw8vSwMvPz9vd1eHj2ubq3+vx4u7y3+ru6vX58vv/8/r99/z/+///+///+///+////f///f///f///f/+/f/+///8/////////f//+///+///+////f/+///+//////3/09Lbx8vW1+Du1N/tw8/busbSs77MrbrIqbbGprTGpbPGn67BmKa5lqS3k6C2k56ylJyzlJuvkpivkpmtipOnipaogo6gg5Cgg5CefY2afo2ddIKUcHuPanWJXWR4Vl1wY2h3YWdyVFphU1le6v/upM6l3P/ZeK928f/y+v/9///+/v/9+//8ys/O//z/3Nvlx8/WxdLUxNDS0N3f2ejr3uzy2Ofq1eHl5PD06vX57fn78vv+9Pz8+f7/+v//+v//+///+///+///+v/9+//+/f/8/f///f///f//+///+v//+v//+//+/f/+/////v3/3eDl19zl2+Tx097sw8/bucXRs7/LrbrIqLXFpbLCo6/Bnau9m6e5mKS2lJ+zk52vkZisjpSnjZKnjJGmiJGliZOli5WnhJGhgY6ceoeVZXKCVmJ0VmF1UVxyVVxwVl1wWFxuWmBtXmNsYGZt7P/v6P/p0//OxfXB8//0/P/9///////+/f/++/39

+/7/+P/87v/z8f/y+v/+/v3///v///7/+vn7/f//0dPTzdLTwczUxNHZ0Nzg2uXp3env5O/309zl6PH67PT78Pj/9/3/9/3/+/7/+v3//f///f///f///f///v7+/f///f//+////f///f///f///f///f//+///+v7/+P//9/3/5O32ztfhxdDevcfYusbYsbrOqbLGp7DEm6S4mqW5mqW5lqK0mKS2l6GylZ+wlZ6sk5yqj5WifoSRen2Lb3SDZm5/XmV4W15zYWJ2YGR2XGFwW2JxWWFuYGVub3d+gIeQd4CNeH+SdXyPTVViVl1m9v/77//z3P/czu/N8/70/f///v3//v3/9/n5+Pr6//7/0dTYwc3TvMrQzdjc1eHj2OTq5fD44ez03ujv6fP68/z/6vP39/7/+f7/+f7/+///+///+////f///f///f///f//+////f///f///f///f///f///f//+///+f7/9/3/5e300tvlyNHev8nauMTWsLrMqrTGoqu/nqi6mKG1nqi6lZ+xmKK0kpytmqKzkpupho+ddHuKb3WCZGp3Y2p5W2J1aG+DcnWKcHOIZGh7WF9wWF9wVVxrWmBrZ253eoCLfISRe4KVcnmMTFJfTlVe6v/toc6j3f/Zeq928f/y+v/++////f/++f/6y87M//3/3t3mxtHVxdLUxs/S0tve2OTq2uju2Obszdnd5vL26fT46vX58vv+8fr99f39+f7/+v//+v//+///+//+/f/+/f///f///////////////////////f//+///+f7/+P7/5+/21+DqydLfwMrbucPUr7nKp7HCpKu+nKS1m6K1m6O0lp6vlp6vmaGykZmqho6ffYaUanOBYmt5Ymt5fIWTiZOlj5isk5ewk5SujpCoiY6jgoecdHqNY2h3YGZzaG57dHuKfYSYeH6RVFhjUVde7P/s6v/n1P/Mx/XA8//y+f/7/f/+/f/8/P/6/f/+